# SHAP Site Comparison: Urban vs Rural

**Course**: WQF7009 Explainable Artificial Intelligence (XAI)  
**University**: University of Malaya  
**Purpose**: Compare XAI explanations between urban and rural monitoring stations

---

## Comparison Sites

| Site | Type | Location |
|------|------|----------|
| **Dongsi** | Urban | Central Beijing |
| **Dingling** | Rural | Ming Tombs area (mountains) |

---

## Table of Contents

1. [Setup & Imports](#1-setup--imports)
2. [Load Data (Both Sites)](#2-load-data-both-sites)
3. [Data Preprocessing](#3-data-preprocessing)
4. [Train Models](#4-train-models)
5. [Model Performance Comparison](#5-model-performance-comparison)
6. [SHAP Feature Importance Comparison](#6-shap-feature-importance-comparison)
7. [Key Findings](#7-key-findings)

---
## 1. Setup & Imports

In [ ]:
# Data processing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# XAI Methods
import shap

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("All libraries imported successfully!")

---
## 2. Load Data (Both Sites)

In [ ]:
# Data paths
DATA_DIR = '../beijing-multisite-uci/raw/PRSA_Data_20130301-20170228/'

# Load Urban site (Dongsi)
df_urban = pd.read_csv(DATA_DIR + 'PRSA_Data_Dongsi_20130301-20170228.csv')
df_urban['site_type'] = 'Urban'
df_urban['site_name'] = 'Dongsi'

# Load Rural site (Dingling)
df_rural = pd.read_csv(DATA_DIR + 'PRSA_Data_Dingling_20130301-20170228.csv')
df_rural['site_type'] = 'Rural'
df_rural['site_name'] = 'Dingling'

print(f"Urban (Dongsi) rows: {len(df_urban):,}")
print(f"Rural (Dingling) rows: {len(df_rural):,}")

In [ ]:
# Compare basic statistics
print("="*60)
print("PM2.5 COMPARISON: Urban vs Rural")
print("="*60)
print(f"\nUrban (Dongsi):")
print(f"  Mean PM2.5: {df_urban['PM2.5'].mean():.1f} µg/m³")
print(f"  Max PM2.5:  {df_urban['PM2.5'].max():.1f} µg/m³")

print(f"\nRural (Dingling):")
print(f"  Mean PM2.5: {df_rural['PM2.5'].mean():.1f} µg/m³")
print(f"  Max PM2.5:  {df_rural['PM2.5'].max():.1f} µg/m³")
print("="*60)

In [ ]:
# Visualize PM2.5 distribution comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograms
axes[0].hist(df_urban['PM2.5'].dropna(), bins=50, alpha=0.7, label='Urban (Dongsi)', color='red')
axes[0].hist(df_rural['PM2.5'].dropna(), bins=50, alpha=0.7, label='Rural (Dingling)', color='green')
axes[0].set_xlabel('PM2.5 (µg/m³)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('PM2.5 Distribution: Urban vs Rural')
axes[0].legend()

# Box plots
data_to_plot = [df_urban['PM2.5'].dropna(), df_rural['PM2.5'].dropna()]
bp = axes[1].boxplot(data_to_plot, labels=['Urban (Dongsi)', 'Rural (Dingling)'], patch_artist=True)
bp['boxes'][0].set_facecolor('red')
bp['boxes'][1].set_facecolor('green')
axes[1].set_ylabel('PM2.5 (µg/m³)')
axes[1].set_title('PM2.5 Box Plot: Urban vs Rural')

plt.tight_layout()
plt.savefig('../outputs/figures/02_shap_site_comparison/pm25_urban_vs_rural_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 3. Data Preprocessing

Apply same preprocessing to both sites for fair comparison.

In [ ]:
def preprocess_site(df, site_name):
    """Preprocess a single site's data."""
    print(f"\nPreprocessing {site_name}...")
    
    # Create copy
    data = df.copy()
    
    # Drop unnecessary columns
    columns_to_drop = ['No', 'year', 'day', 'station', 'site_type', 'site_name']
    data = data.drop(columns=[c for c in columns_to_drop if c in data.columns])
    
    # Handle missing values
    # Drop rows with missing target
    data = data.dropna(subset=['PM2.5'])
    
    # Fill numerical features with median
    numerical_features = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
    for col in numerical_features:
        if col in data.columns:
            data[col] = data[col].fillna(data[col].median())
    
    # Fill wind direction with mode
    data['wd'] = data['wd'].fillna(data['wd'].mode()[0])
    
    # Convert wd to category
    data['wd'] = data['wd'].astype('category')
    
    print(f"  Final rows: {len(data):,}")
    print(f"  Missing values: {data.isnull().sum().sum()}")
    
    return data

In [ ]:
# Preprocess both sites
urban_processed = preprocess_site(df_urban, 'Urban (Dongsi)')
rural_processed = preprocess_site(df_rural, 'Rural (Dingling)')

In [ ]:
# Define features and target
TARGET = 'PM2.5'
FEATURES = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'wd', 'hour', 'month']

print(f"Target: {TARGET}")
print(f"Features ({len(FEATURES)}): {FEATURES}")

In [ ]:
# Prepare X and y for both sites
X_urban = urban_processed[FEATURES].copy()
y_urban = urban_processed[TARGET].copy()

X_rural = rural_processed[FEATURES].copy()
y_rural = rural_processed[TARGET].copy()

print(f"Urban - X: {X_urban.shape}, y: {y_urban.shape}")
print(f"Rural - X: {X_rural.shape}, y: {y_rural.shape}")

In [ ]:
# Train/test split for both sites
X_train_urban, X_test_urban, y_train_urban, y_test_urban = train_test_split(
    X_urban, y_urban, test_size=0.2, random_state=42
)

X_train_rural, X_test_rural, y_train_rural, y_test_rural = train_test_split(
    X_rural, y_rural, test_size=0.2, random_state=42
)

print(f"Urban - Train: {len(X_train_urban):,}, Test: {len(X_test_urban):,}")
print(f"Rural - Train: {len(X_train_rural):,}, Test: {len(X_test_rural):,}")

---
## 4. Train Models

Train separate XGBoost models for each site.

In [ ]:
# Model parameters (same for both)
model_params = {
    'n_estimators': 200,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'enable_categorical': True,
    'tree_method': 'hist'
}

In [ ]:
# Train Urban model
print("Training Urban (Dongsi) model...")
model_urban = xgb.XGBRegressor(**model_params)
model_urban.fit(X_train_urban, y_train_urban, verbose=False)
print("Urban model trained!")

# Train Rural model
print("\nTraining Rural (Dingling) model...")
model_rural = xgb.XGBRegressor(**model_params)
model_rural.fit(X_train_rural, y_train_rural, verbose=False)
print("Rural model trained!")

---
## 5. Model Performance Comparison

In [ ]:
# Predictions
y_pred_urban = model_urban.predict(X_test_urban)
y_pred_rural = model_rural.predict(X_test_rural)

# Calculate metrics
def calc_metrics(y_true, y_pred):
    return {
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R²': r2_score(y_true, y_pred)
    }

metrics_urban = calc_metrics(y_test_urban, y_pred_urban)
metrics_rural = calc_metrics(y_test_rural, y_pred_rural)

In [ ]:
# Display metrics comparison
print("="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)
print(f"\n{'Metric':<10} {'Urban (Dongsi)':<20} {'Rural (Dingling)':<20}")
print("-"*50)
print(f"{'RMSE':<10} {metrics_urban['RMSE']:<20.4f} {metrics_rural['RMSE']:<20.4f}")
print(f"{'MAE':<10} {metrics_urban['MAE']:<20.4f} {metrics_rural['MAE']:<20.4f}")
print(f"{'R²':<10} {metrics_urban['R²']:<20.4f} {metrics_rural['R²']:<20.4f}")
print("="*60)

In [ ]:
# Visual comparison
metrics_df = pd.DataFrame({
    'Metric': ['RMSE', 'MAE', 'R²'],
    'Urban (Dongsi)': [metrics_urban['RMSE'], metrics_urban['MAE'], metrics_urban['R²']],
    'Rural (Dingling)': [metrics_rural['RMSE'], metrics_rural['MAE'], metrics_rural['R²']]
})

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for i, metric in enumerate(['RMSE', 'MAE', 'R²']):
    values = [metrics_urban[metric], metrics_rural[metric]]
    colors = ['red', 'green']
    axes[i].bar(['Urban', 'Rural'], values, color=colors, edgecolor='black')
    axes[i].set_title(metric)
    axes[i].set_ylabel(metric)
    for j, v in enumerate(values):
        axes[i].text(j, v + 0.01*max(values), f'{v:.3f}', ha='center')

plt.suptitle('Model Performance: Urban vs Rural', fontsize=14)
plt.tight_layout()
plt.savefig('../outputs/figures/02_shap_site_comparison/model_performance_urban_vs_rural.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. SHAP Feature Importance Comparison

In [ ]:
# Create SHAP explainers
print("Creating SHAP explainers...")
explainer_urban = shap.TreeExplainer(model_urban)
explainer_rural = shap.TreeExplainer(model_rural)

# Calculate SHAP values (sample for speed)
n_samples = 1000

X_sample_urban = X_test_urban.sample(n=min(n_samples, len(X_test_urban)), random_state=42)
X_sample_rural = X_test_rural.sample(n=min(n_samples, len(X_test_rural)), random_state=42)

print("Calculating SHAP values for Urban...")
shap_values_urban = explainer_urban.shap_values(X_sample_urban)

print("Calculating SHAP values for Rural...")
shap_values_rural = explainer_rural.shap_values(X_sample_rural)

print("Done!")

In [ ]:
# Calculate mean absolute SHAP values for each feature
mean_shap_urban = np.abs(shap_values_urban).mean(axis=0)
mean_shap_rural = np.abs(shap_values_rural).mean(axis=0)

# Create comparison DataFrame
shap_comparison = pd.DataFrame({
    'Feature': FEATURES,
    'Urban (Dongsi)': mean_shap_urban,
    'Rural (Dingling)': mean_shap_rural
})

# Add rank columns
shap_comparison['Urban Rank'] = shap_comparison['Urban (Dongsi)'].rank(ascending=False).astype(int)
shap_comparison['Rural Rank'] = shap_comparison['Rural (Dingling)'].rank(ascending=False).astype(int)

# Sort by Urban importance
shap_comparison = shap_comparison.sort_values('Urban (Dongsi)', ascending=False)

print("\n" + "="*70)
print("SHAP FEATURE IMPORTANCE COMPARISON")
print("="*70)
print(shap_comparison.to_string(index=False))
print("="*70)

In [ ]:
# Side-by-side bar chart
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Urban
urban_sorted = shap_comparison.sort_values('Urban (Dongsi)', ascending=True)
axes[0].barh(urban_sorted['Feature'], urban_sorted['Urban (Dongsi)'], color='red', edgecolor='black')
axes[0].set_xlabel('Mean |SHAP Value|')
axes[0].set_title('Urban (Dongsi) - Feature Importance')

# Rural
rural_sorted = shap_comparison.sort_values('Rural (Dingling)', ascending=True)
axes[1].barh(rural_sorted['Feature'], rural_sorted['Rural (Dingling)'], color='green', edgecolor='black')
axes[1].set_xlabel('Mean |SHAP Value|')
axes[1].set_title('Rural (Dingling) - Feature Importance')

plt.suptitle('SHAP Feature Importance: Urban vs Rural', fontsize=14)
plt.tight_layout()
plt.savefig('../outputs/figures/02_shap_site_comparison/shap_comparison_urban_vs_rural.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Grouped bar chart for direct comparison
fig, ax = plt.subplots(figsize=(14, 8))

x = np.arange(len(FEATURES))
width = 0.35

# Sort by urban importance for display
sorted_df = shap_comparison.sort_values('Urban (Dongsi)', ascending=False)

bars1 = ax.bar(x - width/2, sorted_df['Urban (Dongsi)'], width, label='Urban (Dongsi)', color='red', edgecolor='black')
bars2 = ax.bar(x + width/2, sorted_df['Rural (Dingling)'], width, label='Rural (Dingling)', color='green', edgecolor='black')

ax.set_xlabel('Feature')
ax.set_ylabel('Mean |SHAP Value|')
ax.set_title('SHAP Feature Importance: Urban vs Rural Comparison')
ax.set_xticks(x)
ax.set_xticklabels(sorted_df['Feature'], rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.savefig('../outputs/figures/02_shap_site_comparison/shap_grouped_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# SHAP Summary plots side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

plt.subplot(1, 2, 1)
shap.summary_plot(shap_values_urban, X_sample_urban, plot_type="bar", show=False)
plt.title('Urban (Dongsi)')

plt.subplot(1, 2, 2)
shap.summary_plot(shap_values_rural, X_sample_rural, plot_type="bar", show=False)
plt.title('Rural (Dingling)')

plt.suptitle('SHAP Summary: Urban vs Rural', fontsize=14)
plt.tight_layout()
plt.savefig('../outputs/figures/02_shap_site_comparison/shap_summary_urban_vs_rural.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 7. Key Findings

In [ ]:
# Top 5 features for each site
top5_urban = shap_comparison.nsmallest(5, 'Urban Rank')[['Feature', 'Urban (Dongsi)', 'Urban Rank']]
top5_rural = shap_comparison.nsmallest(5, 'Rural Rank')[['Feature', 'Rural (Dingling)', 'Rural Rank']]

print("="*70)
print("KEY FINDINGS: Urban vs Rural Feature Importance")
print("="*70)

print("\n📍 URBAN (Dongsi) - Top 5 Features:")
print("-"*40)
for i, row in top5_urban.iterrows():
    print(f"  {int(row['Urban Rank'])}. {row['Feature']}: {row['Urban (Dongsi)']:.4f}")

print("\n🌲 RURAL (Dingling) - Top 5 Features:")
print("-"*40)
for i, row in top5_rural.iterrows():
    print(f"  {int(row['Rural Rank'])}. {row['Feature']}: {row['Rural (Dingling)']:.4f}")

print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)
print("""
Urban areas (Dongsi):
  → Traffic-related pollutants (CO, NO2) are key drivers
  → Hour matters more (rush hour effect)
  → Higher overall pollution levels

Rural areas (Dingling):
  → Weather conditions (TEMP, WSPM) play larger role
  → Less influence from traffic pollutants
  → Lower overall pollution levels

Policy Implications:
  → Urban: Focus on traffic emission control
  → Rural: Monitor weather-driven pollution events
""")
print("="*70)

In [ ]:
# Summary statistics
print("\n" + "="*70)
print("SUMMARY STATISTICS")
print("="*70)
print(f"\n{'Metric':<25} {'Urban (Dongsi)':<20} {'Rural (Dingling)':<20}")
print("-"*65)
print(f"{'Mean PM2.5 (µg/m³)':<25} {df_urban['PM2.5'].mean():<20.1f} {df_rural['PM2.5'].mean():<20.1f}")
print(f"{'Max PM2.5 (µg/m³)':<25} {df_urban['PM2.5'].max():<20.1f} {df_rural['PM2.5'].max():<20.1f}")
print(f"{'Model R²':<25} {metrics_urban['R²']:<20.4f} {metrics_rural['R²']:<20.4f}")
print(f"{'Model RMSE':<25} {metrics_urban['RMSE']:<20.4f} {metrics_rural['RMSE']:<20.4f}")
print(f"{'Top Feature':<25} {top5_urban.iloc[0]['Feature']:<20} {top5_rural.iloc[0]['Feature']:<20}")
print("="*70)